In [1]:
import os
import re
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import time as tm
pd.set_option('display.max_row', 1000)
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 1.3 * matplotlib.rcParams['savefig.dpi']

In [2]:
pwd

u'/Users/Jonathan/santander'

In [3]:
# Read data
df_train = pd.read_csv('data/train.csv').dropna(axis=0, how='all').reset_index(drop = True)
df_test = pd.read_csv('data/test.csv').dropna(axis=0, how='all').reset_index(drop = True)

In [4]:
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM

In [5]:
# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

In [6]:
df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

In [7]:
# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

In [8]:
df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

In [9]:
id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

# length of dataset
len_train = len(X_train)
len_test  = len(X_test)

In [10]:
# classifier
clf = xgb.XGBClassifier(missing=np.nan, max_depth=5, n_estimators=350, learning_rate=0.03, nthread=4, subsample=0.95, colsample_bytree=0.85, seed=4242)

X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.3)

# fitting
clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=[(X_eval, y_eval)])

print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))

Will train until validation_0 error hasn't decreased in 20 rounds.
[0]	validation_0-auc:0.819996
[1]	validation_0-auc:0.831046
[2]	validation_0-auc:0.831554
[3]	validation_0-auc:0.819780
[4]	validation_0-auc:0.820737
[5]	validation_0-auc:0.818439
[6]	validation_0-auc:0.806762
[7]	validation_0-auc:0.811631
[8]	validation_0-auc:0.804847
[9]	validation_0-auc:0.806538
[10]	validation_0-auc:0.810069
[11]	validation_0-auc:0.807122
[12]	validation_0-auc:0.806834
[13]	validation_0-auc:0.808827
[14]	validation_0-auc:0.810187
[15]	validation_0-auc:0.812173
[16]	validation_0-auc:0.815095
[17]	validation_0-auc:0.816792
[18]	validation_0-auc:0.817990
[19]	validation_0-auc:0.815265
[20]	validation_0-auc:0.816634
[21]	validation_0-auc:0.818887
[22]	validation_0-auc:0.816132


('Overall AUC:', 0.83953997998434526)


Stopping. Best iteration:
[2]	validation_0-auc:0.831554



In [11]:
# predicting
y_pred= clf.predict_proba(X_test)[:,1]

submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("submission.csv", index=False)

print('Completed!')                

Completed!
